In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
sns.set_theme(style="ticks")

#https://github.com/qpwo/pySCF
from social_choice.profile import Profile#, ballot_box, plurality

#https://github.com/djcunningham0/rankaggregation
import rankaggregation as ra

# Read Submission

In [2]:
f_coocorrence  = "/media/workspace/triplet_session/output/evaluation/MLEvaluationTask/results/SupervisedModelTraining____mars_gym_model_b____06e74a0146_61dcdb92f7_sub_True"
f_most_popular = "/media/workspace/triplet_session/output/evaluation/MLEvaluationTask/results/SupervisedModelTraining____mars_gym_model_b____06e74a0146_d5789bc4c1_sub_True"
f_model        = "/media/workspace/triplet_session/output/evaluation/MLEvaluationTask/results/SupervisedModelTraining____mars_gym_model_b____06e74a0146_80fe3195ca_sub_True"

files = [f_model, f_coocorrence, f_most_popular]
names = ["model", 'coocorrence', 'most_popular']

df_sub = {}
for i in range(len(files)):
    path = files[i]+'/submission_{}.csv'.format(files[i].split("/")[-1])
    _df   = pd.read_csv(path, header=None)
    df_sub[names[i]] = _df

df_sub.keys()

dict_keys(['model', 'coocorrence', 'most_popular'])

In [3]:
df_sub['model'].head()

,0,1,2,3,4,5,6,7,8,9
0,1615991,919498,1117146,187675,1817628,1044975,652718,1154325,92334,2095678
1,228737,1230986,1341158,1659703,106594,855886,1387155,876675,980219,700896
2,1082484,1459575,1081082,1332849,1906937,10243,877640,231634,1154325,678444
3,725371,797984,1716388,286165,1075129,86082,457681,762780,1439187,817689
4,86082,1785970,351968,1030851,1835788,1587422,1920677,1975106,1160594,1114176


In [4]:
df_sub.keys()

dict_keys(['model', 'coocorrence', 'most_popular'])

## Ranked

In [5]:
# import itertools
# import numpy as np
# from collections import OrderedDict

# # list(df_sub.keys())
# def get_list(df_sub, uid, models = []):
#     return list(OrderedDict.fromkeys(list(itertools.chain(*[list(df_sub[k].iloc[uid].values) 
#                                                  for k in models]))))

# candidates = get_list(df_sub, 0, ['model', 'coocorrence', 'most_popular'])
# candidates

In [13]:
agg = ra.RankAggregator()

def get_rerank(reclist, type = 'instant_runoff'):
    if type == 'instant_runoff':
        return agg.instant_runoff(ballots)
    elif type == 'borda':
        l = agg.borda(ballots)
    elif type == 'dowdall':
        l = agg.dowdall(ballots)
    elif type == 'average_rank':
        l = agg.average_rank(ballots)
    
    return [i[0] for i in l]

kind_rerank = 'average_rank'

reranked_list = []
for uid in tqdm(range(len(df_sub['model']))):
    ballots  = [list(df_sub[k].iloc[uid].astype(str).values) for k in ['most_popular','coocorrence', 'model']]
    reranked = list(np.array(get_rerank(ballots, kind_rerank)).astype(int))[:10]
    reranked_list.append(reranked)


100%|██████████| 1000/1000 [00:00<00:00, 1037.41it/s]


## Save

In [14]:
np.savetxt('final_submission.csv', reranked_list, fmt='%i', delimiter=',') 